# 4.6 学習と検証の実施

- 本ファイルでは、OpenPoseの学習と検証の実施を行います。AWSのGPUマシンで計算します。
- p2.xlargeで45分ほどかかります。


# 学習目標

1.	OpenPoseの学習を実装できるようになる

# 事前準備

- これまでの章で実装したクラスと関数をフォルダ「utils」内に用意しています


In [1]:
# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm


In [2]:
# 初期設定
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)


# DataLoader作成

In [3]:
from utils.dataloader import make_datapath_list, DataTransform, COCOkeypointsDataset

# MS COCOのファイルパスリスト作成
train_img_list, train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list = make_datapath_list(
    rootpath="./data/")

# Dataset作成
# 本書ではデータ量の問題から、trainをval_listで作成している点に注意
train_dataset = COCOkeypointsDataset(
    val_img_list, val_mask_list, val_meta_list, phase="train", transform=DataTransform())

# 今回は簡易な学習とし検証データは作成しない
# val_dataset = CocokeypointsDataset(val_img_list, val_mask_list, val_meta_list, phase="val", transform=DataTransform())

# DataLoader作成
batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

# val_dataloader = data.DataLoader(
#    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
# dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}
dataloaders_dict = {"train": train_dataloader, "val": None}


# ネットワークモデル作成

In [4]:
from utils.openpose_net import OpenPoseNet
net = OpenPoseNet()


# 損失関数を定義

In [5]:
# 損失関数の設定
class OpenPoseLoss(nn.Module):
    """OpenPoseの損失関数のクラスです。"""

    def __init__(self):
        super(OpenPoseLoss, self).__init__()

    def forward(self, saved_for_loss, heatmap_target, heat_mask, paf_target, paf_mask):
        """
        損失関数の計算。

        Parameters
        ----------
        saved_for_loss : OpenPoseNetの出力(リスト)

        heatmap_target : [num_batch, 19, 46, 46]
            正解の部位のアノテーション情報

        heatmap_mask : [num_batch, 19, 46, 46]
            heatmap画像のmask

        paf_target : [num_batch, 38, 46, 46]
            正解のPAFのアノテーション情報

        paf_mask : [num_batch, 38, 46, 46]
            PAF画像のmask

        Returns
        -------
        loss : テンソル
            損失の値
        """

        total_loss = 0
        # ステージごとに計算します
        for j in range(6):

            # PAFsとheatmapsにおいて、マスクされている部分（paf_mask=0など）は無視させる
            # PAFs
            pred1 = saved_for_loss[2 * j] * paf_mask
            gt1 = paf_target.float() * paf_mask

            # heatmaps
            pred2 = saved_for_loss[2 * j + 1] * heat_mask
            gt2 = heatmap_target.float()*heat_mask

            total_loss += F.mse_loss(pred1, gt1, reduction='mean') + \
                F.mse_loss(pred2, gt2, reduction='mean')

        return total_loss


criterion = OpenPoseLoss()


# 最適化手法を設定

In [6]:
optimizer = optim.SGD(net.parameters(), lr=1e-2,
                      momentum=0.9,
                      weight_decay=0.0001)


# 学習を実施

In [7]:
# モデルを学習させる関数を作成
use_amp = True
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
device_amp = 'cuda' if torch.cuda.is_available() else 'cpu'

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True
    torch.cuda.empty_cache

    # 画像の枚数
    num_train_imgs = len(dataloaders_dict["train"].dataset)
    batch_size = dataloaders_dict["train"].batch_size

    # イテレーションカウンタをセット
    iteration = 1

    # epochのループ
    for epoch in range(num_epochs):
        torch.cuda.empty_cache

        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                optimizer.zero_grad()
                print('（train）')

            # 今回は検証はスキップ
            else:
                continue
                # net.eval()   # モデルを検証モードに
                # print('-------------')
                # print('（val）')

            # データローダーからminibatchずつ取り出すループ
            for imges, heatmap_target, heat_mask, paf_target, paf_mask in tqdm(dataloaders_dict[phase]):
                # ミニバッチがサイズが1だと、バッチノーマライゼーションでエラーになるのでさける
                # issue #186より不要なのでコメントアウト
                # if imges.size()[0] == 1:
                #     continue
                torch.cuda.empty_cache

                # GPUが使えるならGPUにデータを送る
                imges = imges.to(device)
                heatmap_target = heatmap_target.to(device)
                heat_mask = heat_mask.to(device)
                paf_target = paf_target.to(device)
                paf_mask = paf_mask.to(device)

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    with torch.autocast(device_type=device_amp, dtype=torch.float16, enabled=use_amp):
                        
                        # (out6_1, out6_2)は使わないので _ で代替
                        _, saved_for_loss = net(imges)

                        loss = criterion(saved_for_loss, heatmap_target,
                                        heat_mask, paf_target, paf_mask)
                        del saved_for_loss
                        # 訓練時はバックプロパゲーション
                        if phase == 'train':
                            scaler.scale(loss).backward()
                            scaler.step(optimizer)
                            scaler.update()
                            #optimizer.zero_grad()
                            #loss.backward()
                            #optimizer.step()

                            if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                                t_iter_finish = time.time()
                                duration = t_iter_finish - t_iter_start
                                print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                    iteration, loss.item()/batch_size, duration))
                                t_iter_start = time.time()

                            epoch_train_loss += loss.item()
                            iteration += 1

                        # 検証時
                        # else:
                            #epoch_val_loss += loss.item()

        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss/num_train_imgs, 0))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

    # 最後のネットワークを保存する
    torch.save(net.state_dict(), 'weights/openpose_net_' +
               str(epoch+1) + '.pth')


In [8]:
# 学習・検証を実行する
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-------------
Epoch 1/2
-------------
（train）


  7%|▋         | 10/153 [03:53<52:19, 21.95s/it] 

イテレーション 10 || Loss: 0.0093 || 10iter: 233.3719 sec.


 13%|█▎        | 20/153 [07:52<50:53, 22.96s/it]

イテレーション 20 || Loss: 0.0082 || 10iter: 239.1212 sec.


...

イテレーション 290 || Loss: 0.0014 || 10iter: 355.5072 sec.


 96%|█████████▌| 147/153 [1:24:43<03:24, 34.00s/it]

イテレーション 300 || Loss: 0.0014 || 10iter: 350.8640 sec.


100%|██████████| 153/153 [1:27:47<00:00, 34.43s/it]


-------------
epoch 2 || Epoch_TRAIN_Loss:0.0015 ||Epoch_VAL_Loss:0.0000
timer:  5267.4561 sec.


以上